# Transaction Treatment

Import transactions and clean up to use with systemm

In [2]:
import pandas as pd 
import numpy as np

In [4]:
# import data
raw_trans = pd.read_csv('transactions.csv')
raw_trans.columns = [x.lower() for x in raw_trans.columns]
raw_trans['date'] = pd.to_datetime(raw_trans['date'])
raw_trans.head()

,date,description,original description,amount,transaction type,category,account name,labels,notes
0,2020-09-11,Google,GOOGLE SERVICES,9.99,debit,Advertising,Platinum Card,NaN,NaN
1,2015-05-01,Facebook,FACEBOOK 53WJ28JAG2,21.17,debit,Advertising,CREDIT CARD,NaN,NaN
2,2017-03-13,Google,GOOGLE *Google Music,9.99,debit,Advertising,CREDIT CARD,NaN,NaN
3,2019-05-12,Google,GOOGLE *GOOGLE MUSIC855-836-398A0AXO1XC SELLER...,9.99,debit,Advertising,Blue Cash Everyday,NaN,NaN
4,2019-06-12,Google,GOOGLE *GOOGLE MUSIC855-836-398,9.99,debit,Advertising,Blue Cash Everyday,NaN,NaN


In [13]:
# filter to credit card debits

print ('original length of', len(raw_trans))
filtered_trans = raw_trans.copy()

filtered_trans = filtered_trans.loc[filtered_trans['transaction type']=='debit']
print ('filtered down to', len(filtered_trans), 'after filtering for transaction type')

filtered_trans = filtered_trans.loc[filtered_trans['date']>='2018-01-01']
print ('filtered down to', len(filtered_trans), 'after filtering for dates after beg of 2018')

filtered_trans = filtered_trans.loc[filtered_trans['date']<='2019-12-31']
print ('filtered down to', len(filtered_trans), 'after filtering for dates before end of 2019')

filtered_trans = filtered_trans.loc[~(filtered_trans['account name'].isin(
    ['Essential Checking', 'SAVINGS Account', 'Online Savings']))]
print ('filtered down to', len(filtered_trans), 'after filtering for account name')

original length of 8245
filtered down to 6196 after filtering for transaction type
filtered down to 2243 after filtering for dates after beg of 2018
filtered down to 1741 after filtering for dates before end of 2019
filtered down to 1209 after filtering for account name


In [14]:
# double check accounts
filtered_trans['account name'].value_counts()

CREDIT CARD                                    575
CREDITCARD Account                             220
Blue Cash Everyday                             168
Platinum Card                                  124
Bank of America Cash Rewards Visa Signature    122
Name: account name, dtype: int64

In [15]:
# look for categories to combine
filtered_trans['category'].value_counts()

Restaurants               417
Groceries                 114
Fast Food                 104
Gas & Fuel                 49
Music                      47
Shopping                   43
Rental Car & Taxi          36
Clothing                   32
Utilities                  28
Alcohol & Bars             26
Air Travel                 26
Coffee Shops               26
Pharmacy                   25
Newspapers & Magazines     21
Electronics & Software     21
Hair                       20
Television                 18
Movies & DVDs              17
Entertainment              13
Service Fee                10
Food & Dining              10
Hotel                       9
Business Services           9
Advertising                 8
Gift                        6
Amusement                   5
Travel                      4
Office Supplies             4
Public Transportation       4
Uncategorized               4
                         ... 
Eyecare                     3
Sporting Goods              3
Auto Insur

In [25]:
#create functions to create rudimentary category flags
def create_category_flag(df, cat_list):
    cat_lower = [x.lower() for x in df['category']]
    return flag_helper_vect(cat_lower, cat_list)
    
def flag_helper(cat, cat_list):
    for c in cat_list:
        if c in cat:
            return 1
    return 0
flag_helper_vect = np.vectorize(flag_helper, excluded=[1])

In [33]:
filtered_trans_v2 = filtered_trans.copy()

dining_list = ['restaurant', 'fast food', 'bars', 'dine', 'dining', 'coffee shops']
filtered_trans_v2['dining_flag'] = create_category_flag(filtered_trans_v2, dining_list)

travel_list = ['gas', 'fuel', 'travel', 'hotel', 'transportation']
filtered_trans_v2['travel_flag'] = create_category_flag(filtered_trans_v2, travel_list)

drugstore_list = ['pharmacy']
filtered_trans_v2['drug store_flag'] = create_category_flag(filtered_trans_v2, drugstore_list)

grocery_list = ['grocery', 'groceries', 'super market']
filtered_trans_v2['grocery_flag'] = create_category_flag(filtered_trans_v2, grocery_list)

entertainment_list = ['entertainment', 'movie', 'amusement', 'sports']
filtered_trans_v2['entertainment_flag'] = create_category_flag(filtered_trans_v2, entertainment_list)

In [35]:
filtered_trans_v2.groupby(['dining_flag', 'travel_flag', 'drug store_flag', 'grocery_flag', 'entertainment_flag']).aggregate({'amount':sum})

amount
dining_flag travel_flag drug store_flag grocery_flag entertainment_flag          
0           0           0               0            0                   22889.71
                                                     1                    1521.93
                                        1            0                    2735.23
                        1               0            0                     318.42
            1           0               0            0                    4598.06
1           0           0               0            0                    6183.40

In [49]:
# combine categories into one column
filtered_trans_v2['category_clean'] = 'unclassified'

for flag in ['dining_flag', 'travel_flag', 'drug store_flag', 'grocery_flag', 'entertainment_flag']:
    cat = flag.split('_flag')[0]
    filtered_trans_v2.loc[filtered_trans_v2[flag]==1, 'category_clean'] = cat

filtered_trans_v2['category_clean'].value_counts()

dining           583
unclassified     358
grocery          114
travel            92
entertainment     37
drug store        25
Name: category_clean, dtype: int64

In [50]:
# aggregate to month and flags
filtered_trans_v2['month_year'] = pd.to_datetime([str(d.year) + '-' + str(d.month) for d in filtered_trans_v2['date']])

trans_agg = filtered_trans_v2.groupby(['month_year', 'category_clean']).aggregate({'amount':sum}).reset_index()
trans_agg

,month_year,category_clean,amount
0,2018-01-01,dining,258.44
1,2018-01-01,drug store,3.20
2,2018-01-01,entertainment,258.00
3,2018-01-01,grocery,125.35
4,2018-01-01,travel,554.72
5,2018-01-01,unclassified,553.19
6,2018-02-01,dining,268.64
7,2018-02-01,drug store,31.79
8,2018-02-01,entertainment,76.95
9,2018-02-01,grocery,97.03


In [52]:
# export
trans_agg.to_csv('transactions_clean.csv', index=False)